In [ ]:
import numpy as np 
import pandas as pd 
import os
from glob import glob
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.linear_model import LinearRegression

In [ ]:
train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
test  = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
sample_sub = pd.read_csv('../input/optiver-realized-volatility-prediction/sample_submission.csv')
display(train)
display(test)
display(sample_sub)

Stock ID Act as a kind of unique token for different stock.

Time ID not exact in sequence but kind of specifying a token for time bucket .//A bit Vague

In [ ]:
print('Different Unique Stocks IDS in training ',len(glob('../input/optiver-realized-volatility-prediction/book_train.parquet/*')),
      'Different Unique Stocks IDS in test ',len(glob('../input/optiver-realized-volatility-prediction/book_test.parquet/*'))
     )
print('Uniques in \n',train.nunique())
px.histogram(train.time_id)

Analysis on Train csv First Row 

0	5	0.004136

In [ ]:
def file_path_to_volatility(path,info=False):
    part_data = []
    stock_id = path.split('/')[4].split('=')[1]
    sample_book = pd.read_parquet(path)
    sample_book['wap'] = (sample_book['bid_price1'] * sample_book['ask_size1'] + sample_book['ask_price1'] * sample_book['bid_size1']) / (sample_book['bid_size1']+ sample_book['ask_size1'])
    sample_book.dropna(inplace=True)
    for gid0,gid in sample_book.groupby('time_id'):
        gid['log_return'] = gid['wap'].apply(lambda x:np.log(x)).diff()
        if info :
            print(f'Realized Volatiliy for time id {gid.time_id.iloc[0]} is ' ,np.sqrt(np.sum(gid['log_return'].reset_index(drop=True).drop(index = 0).apply(lambda x:x**2))))
        part_data.append([stock_id,gid.time_id.iloc[0],np.sqrt(np.sum(gid['log_return'].reset_index(drop=True).drop(index = 0).apply(lambda x:x**2)))])
    return part_data
book_dir = glob('../input/optiver-realized-volatility-prediction/book_train.parquet/*/*')
# PRE LOADING THIS FILE (ALREADY RAN AND SAVED IN INPUTS)
# train_data_vol = []
# for i in tqdm(book_dir):
#     train_data_vol.append(file_path_to_volatility(i))
# past_data = pd.concat([pd.DataFrame(i,columns=['stock_id','time_id','vol']) for i in train_data_vol])
# past_data.stock_id = past_data.stock_id.astype('int64')
past_data = pd.read_csv('../input/starter/train_data.csv')
print(past_data.dtypes)
print(train.dtypes)
display(past_data)

In [ ]:
# METRICS
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))
# print('root mean square percentage error'.upper(),rmspe(train_pred['target'],train_pred['vol']),
#      'root mean square error'.upper(),mean_squared_error(train_pred['target'],train_pred['vol']))
print(train.shape,past_data.shape)
display(pd.merge(train,past_data,on=['stock_id','time_id']))

**Try to combine the 2 trade list and make the data suitable for Model **

../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0

../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0

After Merging Trade Book and Trade Train , i will add the train.csv file will give some rows the orignal true volatility and some of them NANS.
First Just considering the orginals one .

In [ ]:
def path_to_data(path):
    """ This return a merged dataframe of trades where the trades actually took place """
#     print(path)
    stock_id = path.split('/')[-1].split('=')[1]
    curr_book = pd.read_parquet(path)
    curr_trade = pd.read_parquet(path.replace('book','trade'))
    merged_data = pd.merge(curr_book,curr_trade,on=['time_id','seconds_in_bucket'])
    merged_data['stock_id'] = stock_id
#     print(curr_book.shape,curr_trade.shape,len(merged_data))
    if len(merged_data) ==0 :
        merged_data = curr_trade.merge(curr_book, how='cross',suffixes=['','_y'])
        merged_data['diff'] = abs(merged_data.seconds_in_bucket-merged_data.seconds_in_bucket_y)
        merged_data = pd.merge(merged_data.groupby(['time_id','seconds_in_bucket'])['diff'].min().reset_index(),merged_data,how="left")
        merged_data.drop(columns=['time_id_y','seconds_in_bucket_y','diff'],inplace=True)
        merged_data['stock_id'] = stock_id
    merged_data.dropna(inplace=True)
    merged_data.reset_index(drop=True)
    return merged_data

def read_all_files(path):
    """ Reads All file in the sub Folder (path / *) and read all parquets (trade/book) and picks only the first occurence based on Stock + Time
        Returns a list of all dataframs use concat to join them back ."""
    demo_all = []
    for i in tqdm(glob(os.path.join(path,'*'))):
        demo_merged = path_to_data(i)
        demo = demo_merged.groupby(['stock_id','time_id']).first().reset_index()
        demo.stock_id = demo.stock_id.astype('int64')
        demo_all.append(demo)
    return demo_all

def files_to_numbers(demo_all,vol_calculated,csv_path = '../input/optiver-realized-volatility-prediction/train.csv'):
    """ Takes in a List of DataFrame and Merges them with a CSV File and then with preprocessed data that we have where we calculate the Volatility
        at end of 10 min or bucket mark """
    csv_file = pd.read_csv(csv_path)
    demo = pd.concat(demo_all).reset_index(drop=True)
    demo_vol = pd.merge(csv_file,demo,on=['stock_id','time_id'])
    demo_vol_all_data = pd.merge(demo_vol,vol_calculated)
    return demo_vol_all_data

def ffill(data_df):
    data_df=data_df.set_index(['time_id', 'seconds_in_bucket'])
    data_df = data_df.reindex(pd.MultiIndex.from_product([data_df.index.levels[0], np.arange(0,600)], names = ['time_id', 'seconds_in_bucket']), method='ffill')
    return data_df.reset_index()

STEPS TO START TRAINING
1. Read All Files , Pick First Occurence by Grouping on stockid time_id
2. Merge With Training csv to give these labels/target and then with preprocessed data that we have where we calculate the Volatility at end of 10 min or bucket mark.
3. This gives a size of 428913 and train csv is of size 428932 . 19 Points have been discarded by us .

In [ ]:
demo_all = read_all_files('../input/optiver-realized-volatility-prediction/book_train.parquet')
data = files_to_numbers(demo_all,past_data,'../input/optiver-realized-volatility-prediction/train.csv')
display(data)
# print(data.stock_id.value_counts())
# print(data.time_id.nunique())

In [ ]:
data_df = data.copy()
feature = ['seconds_in_bucket','bid_price1','ask_price1','bid_price2','ask_price2','bid_size1','ask_size1','bid_size2','ask_size2','price','size','order_count','vol']
data_df.drop(columns=['stock_id','time_id'],inplace=True)
train_data,train_label = data_df[feature],data_df['target']
std_scaler = StandardScaler()
train_data_transform = std_scaler.fit_transform(train_data)



lr = LinearRegression()
lr.fit(train_data_transform,train_label)
print(lr.score(train_data_transform,train_label))
print('Mean Squared Error \t RMSPE METRIC(COMP)')
print(mean_squared_error(lr.predict(train_data_transform),train_label),rmspe(lr.predict(train_data_transform),train_label))
print(lr.predict(train_data_transform[:5]),train_label[:5].values)

LINEAR REGRESSION SCORES .305 - .315 on Test data

To make Prediction on any folder we need 3 things ,
1. Read All files in it 
2. Merge with the Correct CSV File it is related to 
3. Merge with Volatily result ie past data

In [ ]:
demo_all_pred = read_all_files('../input/optiver-realized-volatility-prediction/book_test.parquet')

book_test_dir = glob('../input/optiver-realized-volatility-prediction/book_test.parquet/*/*')
train_data_vol_pred = []
for i in tqdm(book_test_dir):
    train_data_vol_pred.append(file_path_to_volatility(i))
past_data_pred = pd.concat([pd.DataFrame(i,columns=['stock_id','time_id','vol']) for i in train_data_vol_pred])
past_data_pred.stock_id = past_data_pred.stock_id.astype('int64')

data_pred = files_to_numbers(demo_all_pred,past_data_pred,'../input/optiver-realized-volatility-prediction/test.csv')
data_pred.dropna(inplace=True)
data_pred.reset_index(drop=True)
row_orderings = data_pred.row_id
data_pred.drop(columns='row_id',inplace=True)
data_pred = data_pred[feature]
pred = lr.predict(std_scaler.transform(data_pred))
sub = pd.merge(sample_sub,pd.DataFrame({'row_id':row_orderings,'target':pred}),on='row_id',how='left',suffixes=['_old',''])[sample_sub.columns]
sub.target.fillna(0.003048022,inplace=True)
display(sub)
sub.to_csv('submission.csv',index=False)